### 연습
- `ratings_train.txt` 파일 로드
- 결측치 제거
- `document` 컬럼의 문자 정규화 (특수문자 제거, 2칸 이상의 공백 제거, 좌우 공백 제거)
- `document` 컬럼의 중복 제거 + 글자 수가 1개 이하인 행 제거
- DataFrame에서 임의의 sample(`n = 10000`, `random_state = 42`) 로 데이터를 추출하여 저장
</br> `head()` $\rightarrow$ 상위 데이터 | `tail()` $\rightarrow$ 하위 데이터 | `sample()` $\rightarrow$ 무작위 데이터
- train, test 셋으로 8:2 분할
- sbert 모델은 `'BM-K/KoSimCSE-roberta-multitask'` 이용
- Dataset을 정의 (Trainer를 이용하지 않고 Dataset과 DataLoader 사용)
    - 입력받은 document와 label을 document는 SBERT 모델을 이용하여 인코딩
    - label 데이터를 tensor 형태로 변환
    - `__len__` 함수는 라벨의 길이를 되돌려준다.
    - `__getitem__` 함수는 인코딩된 데이터[idx], label[idx]를 되돌려준다
- train, test를 이용해 Dataset를 생성
- `DataLoader`를 이용해 `batch_size = 128`, `shuffle = True`로 구성

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer

---
파일 로드

In [48]:
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [50]:
df.columns

Index(['id', 'document', 'label'], dtype='object')

In [51]:
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [52]:
# 결측치 처리
df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [53]:
df.dropna(subset=['document'], inplace=True)

---
`document` 컬럼의 문자 정규화 (특수문자 제거, 2칸 이상의 공백 제거, 좌우 공백 제거)

In [ ]:
def normalize(text):
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# Series이므로 apply 사용해도 되지만, DataFrame에는 apply 사용 불가능
df['document'] = df['document'].map(normalize)
df['document']

0                                                      굳
1                                   GDNTOPCLASSINTHECLUB
2                 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                       지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4        3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠
                              ...                       
49995            오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네
49999                                         마무리는 또 왜이래
Name: document, Length: 49997, dtype: object

---
`document` 컬럼의 중복 제거 + 글자 수가 1개 이하인 행 제거

In [55]:
df.drop_duplicates(subset=['document'], inplace=True)

In [58]:
flag = df['document'].str.len() > 1
df = df.loc[ flag, ].copy()

---
DataFrame에서 임의의 sample(`n = 10000`, `random_state = 42`) 로 데이터를 추출하여 저장

In [59]:
df_sample = df.sample(n= 10000, random_state= 42).reset_index(drop= True)
df_sample

,id,document,label
0,1387469,바지의 미스터리를 남긴 작품 스판이란 말인가,1
1,8403336,영화보고 이렇게빡치는건처음인거같네ㅋㅋ올레티비 천원으로 반짝할인하길래 내용읽어보니까 ...,0
2,9805002,71년도 원작영화와 별반 차이가 없다.그리고 잔인하고 엽기적이다,0
3,9979651,아 ebs에서 하길래 또 봣는데 역시는 역시다 또 울고말았다 몇번을봐도 눈물 못참는다,1
4,2265984,이딴것도 드라마라고.. 아휴,0
...,...,...,...
9995,4931303,I had so much fun in this movie !! that's good,1
9996,9667858,후속작은 전작과 자동 비교로 덜재미는게 당연시 하는데. 스타트랙은 예외인듯하다. 영...,1
9997,6703982,"춘자라던지, 조폭이라던지 하는 막장 개그씬이 꼭 필요했나 싶다",0
9998,9579629,아이들이 매우 좋아합니다. 처음에는 무서운듯했으나 나중에는 전혀 무섭지 않아요^^,1


---
train, test 셋으로 8:2 분할

In [60]:
train_df, test_df = train_test_split(
    df_sample, test_size= 0.2, random_state= 42, stratify= df_sample['label']
)

In [61]:
train_df['label'].value_counts()

label
0    4066
1    3934
Name: count, dtype: int64

---
sbert 모델은 `'BM-K/KoSimCSE-roberta-multitask'` 이용

In [62]:
model_name = 'BM-K/KoSimCSE-roberta-multitask'
sbert = SentenceTransformer(model_name)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


---
Dataset을 정의 (Trainer를 이용하지 않고 Dataset과 DataLoader 사용)
- `__init__(self, document, labels)`
- 입력받은 document와 labels를 document는 SBERT 모델을 이용하여 인코딩
- labels 데이터를 tensor 형태로 변환
- `__len__(self)` 함수는 라벨의 길이를 되돌려준다.
- `__getitem__(self, idx)` 함수는 인코딩된 데이터[idx], label[idx]를 되돌려준다.

In [63]:
# Dataset을 상속받음
class SBERTDataset(Dataset):
    # 생성자 함수 -> document, labels를 받아와 document는 임베딩, labels는 tensor화
    def __init__(self, document, labels):
        # no_grad(): 자동 미분 일시 정지
        # inference_mode(): 추론 모드 + 미분 정지
        with torch.inference_mode():
            # 로드한 모델을 이용해 인코딩
            self.emb = sbert.encode(
                document,
                convert_to_tensor= True,    # 결괏값을 tensor 형태로 받을 것인가? (False: list 형태)
                normalize_embeddings= True  # L2 정규화할 것인가?
            )
        # labels를 tensor화
        self.labels = torch.tensor(labels, dtype= torch.long)

    def __len__(self):
        # (tensor화된 labels)의 길이를 반환
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.emb[idx], self.labels[idx]

---
train, test를 이용해 Dataset를 생성

In [64]:
# Dataset의 형태로 데이터프레임을 반환
train_ds = SBERTDataset( train_df['document'].tolist(), train_df['label'].tolist() )
test_ds = SBERTDataset( test_df['document'].tolist(), test_df['label'].tolist() )

KeyboardInterrupt: 

---
`DataLoader`를 이용해 `batch_size = 128`, `shuffle = True`로 구성

In [ ]:
# 배치 사이즈 만큼의 데이터를 생성
train_dl = DataLoader(train_ds, batch_size= 128, shuffle= True)
test_dl = DataLoader(test_ds, batch_size= 128, shuffle= True)

train, test 셋 만들었으니 검증, 예측하면 됨